## POS tagging using modified Viterbi

### Data Preparation

In [1]:
#Importing libraries
import nltk
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
import pprint, time

In [2]:
import nltk
nltk.download('treebank')

[nltk_data] Downloading package treebank to
[nltk_data]     C:\Users\DSS7884\AppData\Roaming\nltk_data...
[nltk_data]   Package treebank is already up-to-date!


True

In [3]:
nltk.download('universal_tagset')

[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\DSS7884\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


True

In [4]:
# reading the Treebank tagged sentences
nltk_data = list(nltk.corpus.treebank.tagged_sents(tagset='universal'))

In [5]:
# let's check some of the tagged data
print(nltk_data[1:5])

[[('Mr.', 'NOUN'), ('Vinken', 'NOUN'), ('is', 'VERB'), ('chairman', 'NOUN'), ('of', 'ADP'), ('Elsevier', 'NOUN'), ('N.V.', 'NOUN'), (',', '.'), ('the', 'DET'), ('Dutch', 'NOUN'), ('publishing', 'VERB'), ('group', 'NOUN'), ('.', '.')], [('Rudolph', 'NOUN'), ('Agnew', 'NOUN'), (',', '.'), ('55', 'NUM'), ('years', 'NOUN'), ('old', 'ADJ'), ('and', 'CONJ'), ('former', 'ADJ'), ('chairman', 'NOUN'), ('of', 'ADP'), ('Consolidated', 'NOUN'), ('Gold', 'NOUN'), ('Fields', 'NOUN'), ('PLC', 'NOUN'), (',', '.'), ('was', 'VERB'), ('named', 'VERB'), ('*-1', 'X'), ('a', 'DET'), ('nonexecutive', 'ADJ'), ('director', 'NOUN'), ('of', 'ADP'), ('this', 'DET'), ('British', 'ADJ'), ('industrial', 'ADJ'), ('conglomerate', 'NOUN'), ('.', '.')], [('A', 'DET'), ('form', 'NOUN'), ('of', 'ADP'), ('asbestos', 'NOUN'), ('once', 'ADV'), ('used', 'VERB'), ('*', 'X'), ('*', 'X'), ('to', 'PRT'), ('make', 'VERB'), ('Kent', 'NOUN'), ('cigarette', 'NOUN'), ('filters', 'NOUN'), ('has', 'VERB'), ('caused', 'VERB'), ('a', 'DET

In [6]:
# split data into training and validation set in the ratio 80:20

train_set,test_set = train_test_split(nltk_data,train_size=0.80,test_size=0.20,random_state = 44)

In [7]:
# create list of train and test tagged words
train_tagged = [tup for sent in train_set for tup in sent]
tagged_test = [tup[0] for sent in test_set for tup in sent]
print(len(train_tagged))
print(len(tagged_test))

80126
20550


In [8]:
# checking some of the tagged words.
train_tagged[1:5]

[('city', 'NOUN'), ("'s", 'PRT'), ('Campaign', 'NOUN'), ('Finance', 'NOUN')]

In [9]:
# checking no of unique tags are present in training data
tags = {tag for word,tag in train_tagged}
print(len(tags))
print(tags)

12
{'VERB', 'ADJ', 'NOUN', 'PRON', 'ADP', 'ADV', 'X', 'PRT', 'DET', 'CONJ', '.', 'NUM'}


In [10]:
#checking the no of words are present in vocabulary
vocab = {word for word,tag in train_tagged}
print(len(vocab))

11029


### Build the vanilla Viterbi based POS tagger

#### computing  emission probabilties for a given word

In [11]:
# compute emission probability for a given word for a given tag
def word_given_tag(word,tag,train_bag= train_tagged):
    list_tag = [x for x in train_bag if x[1] == tag]
    tag_count = len(list_tag)    
    w_in_tag = [x[0] for x in list_tag if x[0]==word]    
    word_count_given_tag = len(w_in_tag)    
    return (word_count_given_tag,tag_count)

#### Function to compute transition probabilties for a given tag and previous tag

In [12]:
# compute transition probabilities of a previous and next tag
def t2_given_t1(t2,t1,train_bag=train_tagged):
    tags = [x[1] for x in train_bag]
    t1_tags = [x for x in tags if x==t1]
    count_of_t1 = len(t1_tags)
    t2_given_t1 = [tags[index+1] for index in range(len(tags)-1) if tags[index] == t1 and tags[index+1] == t2]
    count_t2_given_t1 = len(t2_given_t1)
    return(count_t2_given_t1,count_of_t1)

In [13]:
t2_given_t1('NOUN','DET')

(4430, 6906)

In [14]:
# creating transition matrix of tags where each column is t2 and each row is t1
# thus M(i, j) represents P(tj given ti)
matrix_of_tags = np.zeros((len(tags), len(tags)), dtype='float32')
for i, t1 in enumerate(list(tags)):
    for j, t2 in enumerate(list(tags)): 
        matrix_of_tags[i, j] = t2_given_t1(t2, t1)[0]/t2_given_t1(t2, t1)[1]

In [15]:
# converting the matrix to dataframe
tags_df = pd.DataFrame(matrix_of_tags, columns = list(tags), index=list(tags))

In [16]:
tags_df

,VERB,ADJ,NOUN,PRON,ADP,ADV,X,PRT,DET,CONJ,.,NUM
VERB,0.170015,0.065518,0.111869,0.034924,0.090675,0.084224,0.216089,0.030133,0.133616,0.005621,0.034279,0.023037
ADJ,0.013186,0.067506,0.695336,0.000590,0.076757,0.005511,0.022043,0.009841,0.004723,0.016532,0.066326,0.021649
NOUN,0.146882,0.012175,0.262892,0.005044,0.177146,0.016828,0.029133,0.043960,0.013219,0.042091,0.241499,0.009131
PRON,0.486524,0.073550,0.207401,0.008223,0.022385,0.030608,0.096848,0.012791,0.010050,0.005025,0.039287,0.007309
ADP,0.007762,0.107266,0.322306,0.070365,0.017305,0.013233,0.034101,0.001654,0.323578,0.000763,0.038555,0.063112
ADV,0.339377,0.127316,0.033898,0.016555,0.122980,0.081198,0.020102,0.013402,0.073315,0.006307,0.135199,0.030351
X,0.207951,0.015291,0.065176,0.055428,0.141437,0.023700,0.076261,0.186544,0.051414,0.010130,0.163800,0.002867
PRT,0.405405,0.084606,0.248335,0.014884,0.021152,0.010184,0.013709,0.001958,0.096357,0.001958,0.043087,0.058363
DET,0.039096,0.202867,0.641471,0.003620,0.009412,0.012019,0.045613,0.000290,0.005792,0.000434,0.017666,0.021720
CONJ,0.163697,0.118040,0.345768,0.060134,0.051225,0.054566,0.008352,0.005568,0.119154,0.000557,0.033964,0.038975


#### Viterbi Algorithm

1. Given a sequence of words,iterate through the sequence
3. for each word (starting from first word in sequence) calculate the product of emission probabilties and transition probabilties for all possible tags.
4. assign the tag which has maximum probability obtained.
5. move to the next word in sequence and repeat steps 3 and 4.

In [17]:
# Vanilla Viterbi Algorithm
def Viterbi(words, train_bag = train_tagged):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    for key, word in enumerate(words):
        p = [] #initialising list of probability column for a given observation
        for tag in T:
            if key == 0:
                trans_prob = tags_df.loc['.', tag]
            else:
                trans_prob = tags_df.loc[state[-1], tag]   
            # compute emission and state probabilities
            emiss_prob = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_prob = emiss_prob * trans_prob    
            p.append(state_prob) 
        pmax = max(p) # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

#### Testing Vanilla Viterbi Algorithm on sampled test data

In [18]:
random.seed(1234)
# choosing random 5 sentences
x = [random.randint(1,len(test_set)) for x in range(5)]
# list of sents
run_test = [test_set[i] for i in x]
# list of tagged words
run_test_base = [tup for sent in run_test for tup in sent]
# list of untagged words
tagged_test = [tup[0] for sent in run_test for tup in sent]

In [19]:
# tagging the test sentences
start = time.time()
tag_sequence = Viterbi(tagged_test)
end = time.time()
difference = end-start
print("Time taken in seconds: ", difference)
# checking the accuracy of algorithm
check = [i for i, j in zip(tag_sequence, test_run_base) if i == j] 
accuracy = len(check)/len(tag_sequence)
print('Vanilla Viterbi Algorithm Accuracy: ',accuracy*100)

Time taken in seconds:  14.892029047012329
Vanilla Viterbi Algorithm Accuracy:  92.98245614035088


In [20]:
# checking with incorrectly tagged words
[j for i, j in enumerate(zip(tag_sequence, test_run_base)) if j[0] != j[1]]

[(('theaters', 'VERB'), ('theaters', 'NOUN')),
 (('hospitals', 'VERB'), ('hospitals', 'NOUN')),
 (('about', 'ADP'), ('about', 'ADV')),
 (('77,000', 'VERB'), ('77,000', 'NUM')),
 (('free', 'ADJ'), ('free', 'ADV')),
 (('ancient', 'VERB'), ('ancient', 'ADJ')),
 (('New', 'NOUN'), ('New', 'ADJ')),
 (('361.8', 'VERB'), ('361.8', 'NUM'))]

### Solve the problem of unknown words


We can see that all of unknown words have been tagged as 'NUM' as 'NUM' is the first tag in tag list and is assigned
if unknown word is encountered (emission probability =0).

### Viterbi Modification-Techniques (1st technique)

* assign based on transition probabilities only in case of unknown words as emission probability for unknown word is zero.

In [21]:
# use transition probability of tags when emission probability is zero (in case of unknown words)

def Viterbi_1(words, train_bag = train_tagged):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    for key, word in enumerate(words):
        p = []   #list of probability column for a given observation
        p_transition =[] # list for storing transition probabilities
        for tag in T:
            if key == 0:
                prob_trans = tags_df.loc['.', tag]
            else:
                prob_trans = tags_df.loc[state[-1], tag]
            # computing emission and state probabilities
            prob_emiss = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            prob_state = prob_emiss * prob_trans    
            p.append(prob_state)
            p_transition.append(prob_trans)          
        pmax = max(p)
        state_max = T[p.index(pmax)]   
        if(pmax==0):    # if probability is zero (unknown word) then use transition probability
            pmax = max(p_transition)
            state_max = T[p_transition.index(pmax)]
        else:
            state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

In [22]:
# tagging the test sentences
start = time.time() # code to start the timer for execution
tag_sequence = Viterbi_1(tagged_test)
end = time.time() # code to end timer for the execution
difference = end-start
print("Time taken in seconds: ", difference)
check = [i for i, j in zip(tag_sequence, test_run_base) if i == j] 
accuracy = len(check)/len(tag_sequence)
print('Modified Viterbi_1 Accuracy: ',accuracy*100)

Time taken in seconds:  13.930127143859863
Modified Viterbi_1 Accuracy:  94.73684210526315



applying weights based on the probability of tag occurance to the transition probabilities of tags and then use the resulting probability for predicting unknown words. 

In [23]:
probability_of_tag = [] # creating a list containing tuples of POS tags and POS tag occurance probability
total_tag = len([tag for word,tag in train_tagged])
for t in tags:
    each_tag = [tag for word,tag in train_tagged if tag==t]
    probability_of_tag.append((t,len(each_tag)/total_tag))
probability_of_tag

[('VERB', 0.13543668721763222),
 ('ADJ', 0.06341262511544318),
 ('NOUN', 0.2870229388712777),
 ('PRON', 0.027319471831864815),
 ('ADP', 0.09808301924468961),
 ('ADV', 0.03166263135561491),
 ('X', 0.06529715697776003),
 ('PRT', 0.03186231685095974),
 ('DET', 0.08618925192821307),
 ('CONJ', 0.02241469685245738),
 ('.', 0.11661632928138183),
 ('NUM', 0.03468287447270549)]

In [24]:
def Viterbi_1(words, train_bag = train_tagged):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    for key, word in enumerate(words):
        p = []     #initialising list of probability column for a given observation
        p_transition =[] # list for storing transition probabilities
        for tag in T:
            if key == 0:
                prob_trans = tags_df.loc['.', tag]
            else:
                prob_trans = tags_df.loc[state[-1], tag]    
            # computing the emission and state probabilities
            prob_emiss = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            prob_state = prob_emiss * prob_trans    
            p.append(prob_state)
            # find POS tag occurance probability
            tag_p = [pair[1] for pair in probability_of_tag if pair[0]==tag ]
            # calculate the transition prob weighted by tag occurance probability.
            prob_trans = tag_p[0]*prob_trans             
            p_transition.append(prob_trans)
        pmax = max(p)
        state_max = T[p.index(pmax)] 
        # if probability is zero (unknown word) then use weighted transition probability
        if(pmax==0):
            pmax = max(p_transition)
            state_max = T[p_transition.index(pmax)]                                       
        else:
            state_max = T[p.index(pmax)]  
        state.append(state_max)
    return list(zip(words, state))

In [25]:
# tagging the test sentences
start = time.time()
tag_sequence = Viterbi_1(tagged_test)
end = time.time()
difference = end-start
print("Time taken in seconds: ", difference)
check = [i for i, j in zip(tag_sequence, test_run_base) if i == j] 
accuracy = len(check)/len(tag_sequence)
print('Modified Viterbi_1 Accuracy: ',accuracy*100)

Time taken in seconds:  14.055493116378784
Modified Viterbi_1 Accuracy:  94.73684210526315


**Compared to previous viterbi alogorithm, we observe that better accuracy is obtained by using weighted transition probabilties.**

In [26]:
# let's check the incorrectly tagged words
[j for i, j in enumerate(zip(tag_sequence, test_run_base)) if j[0] != j[1]]

[(('about', 'ADP'), ('about', 'ADV')),
 (('77,000', 'NOUN'), ('77,000', 'NUM')),
 (('free', 'ADJ'), ('free', 'ADV')),
 (('ancient', 'NOUN'), ('ancient', 'ADJ')),
 (('New', 'NOUN'), ('New', 'ADJ')),
 (('361.8', 'NOUN'), ('361.8', 'NUM'))]

The following list of words have been **correctly POS tagged by Viterbi_1** as compared to vanilla Viterbi Algorithm:

* about:correctly tagged as ADV
* 77,000:correctly tagged as NUM
* free: correctly tagged as ADV
* 361.8: correctly tagged as NUM

### Viterbi Modification-Technique II

**Alternative solution for unknown words:** 

*we further observe that POS tag 'X' can be easily encapsulated in regex rule, so we extract it only based on ruled based tagger.*

Let's define a rule based tagger as below:

In [27]:
# specify patterns for tagging
patterns = [
    (r'.*s$', 'NOUN'),                # plural nouns
    (r'\*T?\*?-[0-9]+$', 'X'),        # X
    (r'^-?[0-9]+(.[0-9]+)?$', 'NUM'), # cardinal numbers
    (r'.*', 'NOUN')                   # nouns
    (r'.*ing$', 'VERB'),              # gerund
    (r'.*ed$', 'VERB'),               # past tense 
    (r'.*es$', 'VERB'),               # verb    
    (r'.*\'s$', 'NOUN'),              # possessive nouns
]
# rule based tagger
rule_based_tagger = nltk.RegexpTagger(patterns)

In [28]:
#Backing off to rule based tagger in case unknown word is encountered.
def Viterbi_2(words, train_bag = train_tagged):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    for key, word in enumerate(words):
        p = []      #initialise list of probability column for a given observation
        for tag in T:
            if key == 0:
                prob_trans = tags_df.loc['.', tag]
            else:
                prob_trans = tags_df.loc[state[-1], tag]  
            # computing emission and state probabilities
            prob_emiss = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            prob_state = prob_emiss * prob_trans    
            p.append(prob_state)    
        pmax = max(p)
        state_max = rule_based_tagger.tag([word])[0][1]       
        if(pmax==0):
            state_max = rule_based_tagger.tag([word])[0][1] # assign based on rule based tagger
        else:
            if state_max != 'X':
                # getting state for which probability is maximum
                state_max = T[p.index(pmax)]                
        state.append(state_max)
    return list(zip(words, state))

In [29]:
# tagging the test sentences
start = time.time()
tag_sequence = Viterbi_2(tagged_test)
end = time.time()
difference = end-start
print("Time taken in seconds: ", difference)

Time taken in seconds:  14.0182785987854


In [30]:
# accuracy
check = [i for i, j in zip(tag_sequence, test_run_base) if i == j] 
accuracy = len(check)/len(tag_sequence)
print('Modified Viterbi_2 Accuracy: ',accuracy*100)

Modified Viterbi_2 Accuracy:  95.6140350877193


In [31]:
# let's check the incorrectly tagged words
[j for i, j in enumerate(zip(tag_sequence, test_run_base)) if j[0]!=j[1]]

[(('bans', 'NOUN'), ('bans', 'VERB')),
 (('about', 'ADP'), ('about', 'ADV')),
 (('free', 'ADJ'), ('free', 'ADV')),
 (('ancient', 'NOUN'), ('ancient', 'ADJ')),
 (('New', 'NOUN'), ('New', 'ADJ'))]

The following list of words have been correctly POS tagged by Viterbi_2 as compared to vanilla Viterbi Algorithm:

* bans:correctly tagged as verb
* about:correctly tagged as ADV
* free: correctly tagged as ADV
* New: correctly tagged as ADJ

As We know that the rule based tagger assigns 'NOUN' by default if word does not fall in any rule, to correct this let's assign the tags for any such word based purely on transition probability of tags. for this to happen,first we will modify the rule based tagger to output 'NN' instead of 'NOUN' in case word does not satisfy any rules. We also observe that any capitalized word can still be defaulted as 'NOUN' so will add one more rule for that case.

In [32]:
# specify patterns for tagging
patterns = [
    (r'.*s$', 'NOUN'),                # plural nouns
    (r'\*T?\*?-[0-9]+$', 'X'),        # X
    (r'^-?[0-9]+(.[0-9]+)?$', 'NUM'), # cardinal numbers
    (r'^[A-Z][a-z].*', 'NOUN'),       # NOUN
    (r'.*', 'NN')                     # default
    (r'.*ing$', 'VERB'),              # gerund
    (r'.*ed$', 'VERB'),               # past tense 
    (r'.*es$', 'VERB'),               # verb    
    (r'.*\'s$', 'NOUN'),              # possessive nouns
]
# rule based tagger
rule_based_tagger = nltk.RegexpTagger(patterns)

In [33]:
# modified version of Viterbi algorithm
def Viterbi_2(words, train_bag = train_tagged):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    for key, word in enumerate(words):
        p = []    #initialise list of probability column for a given observation
        p_transition =[] # for storing transition probabilities
        for tag in T:
            if key == 0:
                prob_trans = tags_df.loc['.', tag]
            else:
                prob_trans = tags_df.loc[state[-1], tag]
            # compute emission and state probabilities
            prob_emiss = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            prob_state = prob_emiss * prob_trans    
            p.append(prob_state) 
            # find POS tag occurance probability
            tag_p = [pair[1] for pair in probability_of_tag if pair[0]==tag ]
            # calculate the transition prob weighted by tag occurance probability.
            prob_trans = tag_p[0]*prob_trans
            p_transition.append(prob_trans)
        pmax = max(p)
        state_max = rule_based_tagger.tag([word])[0][1] 
        # obtaining the state for which probability is maximum
        if(pmax==0):
            state_max = rule_based_tagger.tag([word])[0][1] # assign based on rule based tagger
            # if unknown word does not satisfy any rule, find the tag with maximum transition probability
            if state_max == 'NN':
                pmax = max(p_transition)
                state_max = T[p_transition.index(pmax)]                      
        else:
             if state_max != 'X':
                # getting state for which probability is maximum
                state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

In [34]:
# tagging the test sentences
start = time.time()
tag_sequence = Viterbi_2(tagged_test)
end = time.time()
difference = end-start
print("Time taken in seconds: ", difference)

Time taken in seconds:  14.236104249954224


In [35]:
# checking the accuracy
check = [i for i, j in zip(tag_sequence, test_run_base) if i == j] 
accuracy = len(check)/len(tag_sequence)
print('Modified Viterbi Algorithm Accuracy: ',accuracy*100)

Modified Viterbi Algorithm Accuracy:  95.6140350877193


In [36]:
# let's check the incorrectly tagged words
[j for i, j in enumerate(zip(tag_sequence, test_run_base)) if j[0] != j[1]]

[(('bans', 'NOUN'), ('bans', 'VERB')),
 (('about', 'ADP'), ('about', 'ADV')),
 (('free', 'ADJ'), ('free', 'ADV')),
 (('ancient', 'NOUN'), ('ancient', 'ADJ')),
 (('New', 'NOUN'), ('New', 'ADJ'))]

the following list of words has been correctly tagged by Viterbi_2 as compared to Viterbi_1
* bans:correctly tagged as verb
* about:correctly tagged as ADV
* free: correctly tagged as ADV
* New: correctly tagged as ADJ

#### Evaluating tagging accuracy

#### Evaluate vanilla Viterbi and modified Viterbi on entire test data

In [37]:
tagged_test = [tup[0] for sent in test_set for tup in sent]
test_run_base = [tup for sent in test_set for tup in sent]

In [38]:
# tagging the test sentences for Viterbi (rule-based)
start = time.time()
tag_sequence = Viterbi(tagged_test)
end = time.time()
difference = end-start
print("Time taken in seconds: ", difference)
# accuracy
check = [i for i, j in zip(tag_sequence, test_run_base) if i == j] 
accuracy = len(check)/len(tag_sequence)
print('Modified Viterbi Algorithm Accuracy: ',accuracy*100)

Time taken in seconds:  2447.593504190445
Modified Viterbi Algorithm Accuracy:  91.31386861313868


In [39]:
# tagging the test sentences (after modification)
start = time.time()
tag_sequence = Viterbi_2(tagged_test)
end = time.time()
difference = end-start

print("Time taken in seconds: ", difference)

# accuracy
check = [i for i, j in zip(tag_sequence, test_run_base) if i == j] 
accuracy = len(check)/len(tag_sequence)
print('Modified Viterbi Algorithm Accuracy: ',accuracy*100)

Time taken in seconds:  2744.008337020874
Modified Viterbi Algorithm Accuracy:  95.0316301703163


#### Evaluating tagging on sample 'Test_sentences.txt' file

In [57]:
f = open('Test_sentences.txt')

In [58]:
text = f.read()
sample_test_sent = text.splitlines()
f.close()

In [74]:
# list of untagged words
sample_tokens = [word for sent in sample_test_sent for word in sent.split()]

In [75]:
sample_tokens 

['Android',
 'is',
 'a',
 'mobile',
 'operating',
 'system',
 'developed',
 'by',
 'Google.',
 'Android',
 'has',
 'been',
 'the',
 'best-selling',
 'OS',
 'worldwide',
 'on',
 'smartphones',
 'since',
 '2011',
 'and',
 'on',
 'tablets',
 'since',
 '2013.',
 'Google',
 'and',
 'Twitter',
 'made',
 'a',
 'deal',
 'in',
 '2015',
 'that',
 'gave',
 'Google',
 'access',
 'to',
 "Twitter's",
 'firehose.',
 'Twitter',
 'is',
 'an',
 'online',
 'news',
 'and',
 'social',
 'networking',
 'service',
 'on',
 'which',
 'users',
 'post',
 'and',
 'interact',
 'with',
 'messages',
 'known',
 'as',
 'tweets.',
 'Before',
 'entering',
 'politics,',
 'Donald',
 'Trump',
 'was',
 'a',
 'domineering',
 'businessman',
 'and',
 'a',
 'television',
 'personality.',
 'The',
 '2018',
 'FIFA',
 'World',
 'Cup',
 'is',
 'the',
 '21st',
 'FIFA',
 'World',
 'Cup,',
 'an',
 'international',
 'football',
 'tournament',
 'contested',
 'once',
 'every',
 'four',
 'years.',
 'This',
 'is',
 'the',
 'first',
 'World',

In [76]:
# tagging the test sentences
start = time.time()
sample_tag_sequence = Viterbi(sample_tokens)
end = time.time()
difference = end-start
print("Time taken in seconds: ", difference)

Time taken in seconds:  24.36722493171692


In [77]:
sample_tag_sequence

[('Android', 'VERB'),
 ('is', 'VERB'),
 ('a', 'DET'),
 ('mobile', 'ADJ'),
 ('operating', 'NOUN'),
 ('system', 'NOUN'),
 ('developed', 'VERB'),
 ('by', 'ADP'),
 ('Google.', 'VERB'),
 ('Android', 'VERB'),
 ('has', 'VERB'),
 ('been', 'VERB'),
 ('the', 'DET'),
 ('best-selling', 'ADJ'),
 ('OS', 'VERB'),
 ('worldwide', 'VERB'),
 ('on', 'ADP'),
 ('smartphones', 'VERB'),
 ('since', 'ADP'),
 ('2011', 'VERB'),
 ('and', 'CONJ'),
 ('on', 'ADP'),
 ('tablets', 'NOUN'),
 ('since', 'ADP'),
 ('2013.', 'VERB'),
 ('Google', 'VERB'),
 ('and', 'CONJ'),
 ('Twitter', 'VERB'),
 ('made', 'VERB'),
 ('a', 'DET'),
 ('deal', 'NOUN'),
 ('in', 'ADP'),
 ('2015', 'VERB'),
 ('that', 'ADP'),
 ('gave', 'VERB'),
 ('Google', 'VERB'),
 ('access', 'NOUN'),
 ('to', 'PRT'),
 ("Twitter's", 'VERB'),
 ('firehose.', 'VERB'),
 ('Twitter', 'VERB'),
 ('is', 'VERB'),
 ('an', 'DET'),
 ('online', 'VERB'),
 ('news', 'NOUN'),
 ('and', 'CONJ'),
 ('social', 'ADJ'),
 ('networking', 'NOUN'),
 ('service', 'NOUN'),
 ('on', 'ADP'),
 ('which', 'D

We can see that several words have been misclassified by vanilla Viterbi POS tagger, for example:
* Android as VERB
* has as verb
* 2015 as verb
* 2011 as verb

In [78]:
# tagging the test sentences
start = time.time()
sample_tag_sequence = Viterbi_2(sample_tokens)
end = time.time()
difference = end-start
print("Time taken in seconds: ", difference)

Time taken in seconds:  22.67865014076233


In [79]:
sample_tag_sequence

[('Android', 'NOUN'),
 ('is', 'VERB'),
 ('a', 'DET'),
 ('mobile', 'ADJ'),
 ('operating', 'NOUN'),
 ('system', 'NOUN'),
 ('developed', 'VERB'),
 ('by', 'ADP'),
 ('Google.', 'NOUN'),
 ('Android', 'NOUN'),
 ('has', 'VERB'),
 ('been', 'VERB'),
 ('the', 'DET'),
 ('best-selling', 'ADJ'),
 ('OS', 'NOUN'),
 ('worldwide', 'NOUN'),
 ('on', 'ADP'),
 ('smartphones', 'VERB'),
 ('since', 'ADP'),
 ('2011', 'NUM'),
 ('and', 'CONJ'),
 ('on', 'ADP'),
 ('tablets', 'NOUN'),
 ('since', 'ADP'),
 ('2013.', 'NOUN'),
 ('Google', 'NOUN'),
 ('and', 'CONJ'),
 ('Twitter', 'NOUN'),
 ('made', 'VERB'),
 ('a', 'DET'),
 ('deal', 'NOUN'),
 ('in', 'ADP'),
 ('2015', 'NUM'),
 ('that', 'ADP'),
 ('gave', 'VERB'),
 ('Google', 'NOUN'),
 ('access', 'NOUN'),
 ('to', 'PRT'),
 ("Twitter's", 'NOUN'),
 ('firehose.', 'NOUN'),
 ('Twitter', 'NOUN'),
 ('is', 'VERB'),
 ('an', 'DET'),
 ('online', 'NOUN'),
 ('news', 'NOUN'),
 ('and', 'CONJ'),
 ('social', 'ADJ'),
 ('networking', 'NOUN'),
 ('service', 'NOUN'),
 ('on', 'ADP'),
 ('which', 'DET

All these cases were correctly POS tagged by Viterbi_2:
* Android as NOUN
* 2011 as NUM
* 2018 as NUM

### Compare the tagging accuracies of the modifications with the vanilla Viterbi algorithm

Vanilla Viterbi Algorithm Accuracy:  **92.98245614035088**

Modified Viterbi Algorithm Accuracy:  **95.0316301703163**

### List down cases which were incorrectly tagged by original POS tagger and got corrected by your modifications

* about:correctly tagged as ADV
* free: correctly tagged as ADV
* New: correctly tagged as ADJ
* Android: correctly tagged as NOUN
* Google: correctly tagged as NOUN
* 2011 : correctly tagged as NUM